## Context
This is image data of Natural Scenes around the world.

## Content
This Data contains around 25k images of size 150x150 distributed under 6 categories.<br>
{'buildings' -> 0,<br>
'forest' -> 1,<br>
'glacier' -> 2,<br>
'mountain' -> 3,<br>
'sea' -> 4,<br>
'street' -> 5 }

The Train, Test and Prediction data is separated in each zip files. There are around 14k images in Train, 3k in Test and 7k in Prediction.

### Import necessary Libraries

In [1]:
import numpy as np 
import pandas as pd 
import os 
import torch 
import torchvision
import matplotlib.pyplot as plt 
import torch.nn as nn 
import torch.nn.functional as F 
from torchvision.transforms import ToTensor 
import torchvision.transforms as tt 
from torchvision.utils import make_grid 
from torch.utils.data.dataloader import DataLoader 
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split 
%matplotlib inline 

In [2]:
print(torch.__version__)

print(torchvision.__version__)

print(torch.version.cuda)

2.3.1+cu121
0.18.1+cu121
12.1


In [3]:
train = ImageFolder("/config/workspace/Data/seg_train",transform=tt.Compose([
    tt.Resize(64),
    tt.RandomCrop(64),
    tt.ToTensor(),
]))

In [4]:
train.classes

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

In [5]:
train_dl = DataLoader(train,64,shuffle=True,num_workers=3,pin_memory=True)

In [6]:
def get_mean_std(dl):
    sum_,sqaured_sum,batches = 0,0,0
    for data, _ in dl:
        sum_ += torch.mean(data,dim=([0,2,3]))
        sqaured_sum += torch.mean(data**2,dim=([0,2,3]))
        batches += 1

    mean = sum_/batches
    std = (sqaured_sum/batches - mean**2)**0.5
    return mean,std 

mean,std = get_mean_std(train_dl)
mean,std 

(tensor([0.4302, 0.4575, 0.4539]), tensor([0.2481, 0.2467, 0.2806]))